In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('extracted_RC_2022-11.csv0.csv')
data = data.dropna(subset = ['body'])
data = data.loc[0:500, :]
data

,author,author_fullname,created_utc,id,parent_id,body,score,subreddit,subreddit_id
0,LordoftheSynth,t2_4clv4,1667260800,iukc2dg,t1_iuhuhjh,/r/FuckOff,0,InfrastructurePorn,t5_2sss3
1,RedditMP4Bot,t2_brd1xvt2,1667260800,iukc2dh,t1_iukc00y,## [**Download MP4**](https://redditmp4.com/yi...,1,inniepenetration,t5_2a3are
2,metooneither,t2_85nmpvr4,1667260800,iukc2dj,t3_yiaxrr,Was a democrat. Came to the realization that ...,1,AskReddit,t5_2qh1i
3,ibeanez_seattle,t2_eq9qmvhx,1667260800,iukc2dp,t1_iuibhkr,Yes,2,egg_irl,t5_3n3op
4,scaredycat_z,t2_21qo5wqc,1667260800,iukc2du,t3_yijp7l,Orthodox Jew here. I'll say what no-one else h...,4,Rockland,t5_2r9f7
...,...,...,...,...,...,...,...,...,...
496,Trash_McTrasherson,t2_m3r7n,1667260819,iukc3u3,t3_yiqxiv,Crack open a beer. This party is just getting ...,2,florida,t5_2qh7q
497,AutoModerator,t2_6l4z3,1667260819,iukc3u6,t3_yit31n,\n\nWelcome to /r/pregnant! This is a space fo...,1,pregnant,t5_2qwzb
498,VengefulKangaroo,t2_idtas,1667260819,iukc3u7,t1_iuj4f52,I think if Dunsparce is exclusive it will be w...,3,PokeLeaks,t5_2ylx9
499,Doesanybodylikestuff,t2_1ahtjwye,1667260819,iukc3u9,t3_yie3x7,Omg that guy is phenomenal.,1,worldnewsvideo,t5_31m0v


In [3]:
sites = pd.read_excel('news_sites.xlsx')
sites

,domain,fake,lowcred,reputable,satire
0,.k12.ca.us,0,0,1,0
1,.k12.or.us,0,0,1,0
2,100percentfedup.com,1,0,0,0
3,1011now.com,0,0,1,0
4,10news.com,0,0,1,0
...,...,...,...,...,...
10957,zootfeed.com,1,0,0,0
10958,zoriah.com,0,0,1,0
10959,zoriah.net,0,0,1,0
10960,zougla.gr,0,0,1,0


In [4]:
def match_sites(x, sites):
    for i in sites.index:
        domain = sites.loc[i, 'domain']
        if domain == x:
            return sites.loc[i, :]
    
    #Return empty seires if not find
    return pd.Series()

In [6]:
new_data = pd.DataFrame(columns = data.columns)
url = []
domain = []
fake = []
lowcred = []
reputable = []
satire = []

for i in data.index:
    temp_url = []
    temp_domain = []
    temp_fake = []
    temp_lowcred = []
    temp_reputable = []
    temp_satire = []
    
    #Find urls in one row
    t = data.loc[i, 'body'].split()
    for j in t:
        match = re.search(r'https', j)
        if match:
            #Remove unnecessay characters in the beginning and in the end
            #Check the beginning
            a = j.split('https:')
            a = 'https:' + a[1]
            
            #Check ")" in the end
            b = a.split(')')
            b = b[0]
            
            #Check "\n" in the end
            c = b.split('\n')
            c = c[0]
        
            temp_url.append(c)
    
    #Remove row if url doesn't match with any site
    temp_remove = []
    for k in temp_url:
        x = k.split('/')[2]
        #Remove www.
        if x[0:4] == 'www.':
            x = x[4:]
        find = match_sites(x, sites)
        if find.empty:
            temp_remove.append(k)
        else:
            temp_domain.append(find['domain'])
            temp_fake.append(find['fake'])
            temp_lowcred.append(find['lowcred'])
            temp_reputable.append(find['reputable'])
            temp_satire.append(find['satire'])
            
    for l in temp_remove:
        temp_url.remove(l)
        
    url.extend(temp_url)
    domain.extend(temp_domain)
    fake.extend(temp_fake)
    lowcred.extend(temp_lowcred)
    reputable.extend(temp_reputable)
    satire.extend(temp_satire)
    #For multiple columns, add additional rows
    for m in range(len(temp_url)):
        new_data = new_data.append(data.loc[i, :], ignore_index = True)
        
new_data['url'] = url
new_data['domain'] = domain
new_data['fake'] = fake
new_data['lowcred'] = lowcred
new_data['reputable'] = reputable
new_data['satire'] = satire
new_data

C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\U

C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2159084960.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(data.loc[i, :], ignore_index = True)
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2159084960.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(data.

C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2159084960.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(data.loc[i, :], ignore_index = True)
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  retur

C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2444419429.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2159084960.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(data.loc[i, :], ignore_index = True)
C:\Users\chris\AppData\Local\Temp\ipykernel_24776\2159084960.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(data.

,author,author_fullname,created_utc,id,parent_id,body,score,subreddit,subreddit_id,url,domain,fake,lowcred,reputable,satire
0,AutoModerator,t2_6l4z3,1667260800,iukc2f3,t3_yit2jt,Welcome to JerkBuds u/Shadysoldiers! Stay GOON...,1,JerkBuds,t5_2vb08b,https://www.reddit.com/r/JerkBuds/about/rules,reddit.com,0,0,1,0
1,AutoModerator,t2_6l4z3,1667260800,iukc2f3,t3_yit2jt,Welcome to JerkBuds u/Shadysoldiers! Stay GOON...,1,JerkBuds,t5_2vb08b,https://www.reddit.com/message/compose?to=/r/j...,reddit.com,0,0,1,0
2,AutoModerator,t2_6l4z3,1667260800,iukc2fh,t3_yit2k1,**Hey there! Have you checked out these resour...,1,wownoob,t5_35idp,https://www.reddit.com/r/wownoob/about/rules/,reddit.com,0,0,1,0
3,AutoModerator,t2_6l4z3,1667260801,iukc2i2,t3_yit2km,Thanks for posting. We are Happy to see you on...,1,bigtitspics,t5_3owlq,https://www.reddit.com/message/compose?to=/r/B...,reddit.com,0,0,1,0
4,AutoModerator,t2_6l4z3,1667260802,iukc2jq,t3_yit2kv,This post has been flaired as a money transfer...,1,beermoneyuk,t5_359qr,https://www.reddit.com/message/compose/?to=/r/...,reddit.com,0,0,1,0
5,AutoModerator,t2_6l4z3,1667260803,iukc2lb,t1_iukc2jm,Hello /u/Last_Result7808 -\n\nYour [comment](h...,1,firstimpression,t5_2sg6k,https://www.reddit.com/r/firstimpression/comme...,reddit.com,0,0,1,0
6,AutoModerator,t2_6l4z3,1667260803,iukc2nh,t3_yit2lo,"Hey /u/Geekykitty9, thanks for the post!\n\nTh...",1,BonerMaterial,t5_2ucwa,https://www.reddit.com/r/BonerMaterial/about/r...,reddit.com,0,0,1,0
7,AutoModerator,t2_6l4z3,1667260804,iukc2o0,t3_yit2ly,Looking for a new sportbook? Check out our rec...,1,sportsbetting,t5_2s443,https://www.reddit.com/r/sportsbetting/comment...,reddit.com,0,0,1,0
8,AutoModerator,t2_6l4z3,1667260804,iukc2o0,t3_yit2ly,Looking for a new sportbook? Check out our rec...,1,sportsbetting,t5_2s443,https://www.reddit.com/r/sportsbetting/comment...,reddit.com,0,0,1,0
9,AutoModerator,t2_6l4z3,1667260804,iukc2o6,t3_yit2lp,Hola\ndesde el 6 de junio tenemos nuevas regla...,1,Colombia,t5_2qrpc,https://www.reddit.com/r/Colombia/comments/v62...,reddit.com,0,0,1,0
